#### _Programming Exercise 2.2.2:_  Signing with a tweaked 2-of-2 MuSig key pair

In [ ]:
# Generate key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Create an aggregate MuSig pubkey
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Apply challenge factors to keys
privkey1_c = privkey1 * c_map[pubkey1]
privkey2_c = privkey2 * c_map[pubkey2]
pubkey1_c = pubkey1 * c_map[pubkey1]
pubkey2_c = pubkey2 * c_map[pubkey2]

# Negate if needed
if agg_pubkey.get_y()%2 != 0:
    agg_pubkey.negate()
    privkey1_c.negate()
    privkey2_c.negate()
    pubkey1_c.negate()
    pubkey2_c.negate()

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = random.randrange(1, SECP256K1_ORDER)
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)

# Nonce generation & aggregation
# Remember to negate the individual nonce values if required
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces()
k1 = generate_schnorr_nonce()
k2 = generate_schnorr_nonce()
R_agg, negated = aggregate_schnorr_nonces([k1.get_pubkey(), k2.get_pubkey()])
if negated:
    k1.negate()
    k2.negate()

# Signing and signature aggregation
msg = sha256(b'msg')

# Sign individually and then aggregate partial signatures. A factor (e * tweak)
# needs to be added to the list of partial signatures
# Method: sign_musig(private_key, nonce_key, nonce_point, public_key, msg)
# Method: aggregate_musig_signatures(partial_signature_list, aggregate nonce)
e = musig_digest(R_agg, agg_pubkey_tweaked, msg)
s1 = sign_musig(privkey1_c, k1, R_agg, agg_pubkey_tweaked, msg)
s2 = sign_musig(privkey2_c, k2, R_agg, agg_pubkey_tweaked, msg)
sig_agg = aggregate_musig_signatures([s1,s2,(e*tweak)], R_agg)

assert agg_pubkey_tweaked.verify_schnorr(sig_agg, msg)
print("Success!")

#### _Programming Exercise 2.2.6:_ Construct taproot output with tweaked public key

In [ ]:
# Example key pair
privkey = ECKey().set(102118636618570133408735518698955378316807974995033705330357303547139065928052)
internal_pubkey = privkey.get_pubkey()

if internal_pubkey.get_y()%2 != 0:
    privkey.negate()
    internal_pubkey.negate()

# Example tweak
taptweak = bytes.fromhex('2a2fb476ec9962f262ff358800db0e7364287340db73e5e48db36d1c9f374e30')

# Tweak the private key
# Method: ECKey.add()
tweaked_privkey = privkey.add(taptweak)

# Tweak the public key
# Method: use tweak_add()
taproot_pubkey = internal_pubkey.tweak_add(taptweak)
taproot_pubkey_b = taproot_pubkey.get_bytes()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
address = program_to_witness(0x01, taproot_pubkey_b)

assert address == "bcrt1pjnux0f7037ysqv2aycfntus0t606sjyu0qe2xqewlmhulpdujqeq2z4st9"
print("Success! Address: {}".format(address))

#### _Programming Exercise 2.2.9:_ Spend taproot output with key path

In [ ]:
# Sign transaction with tweaked private key
# Method: TaprootSignatureHash(tx, output_list, hash_type=int, input_index=int, scriptpath=bool)
sighash = TaprootSignatureHash(spending_tx, [tx.vout[0]], 0x0, 0, False)
sig = tweaked_privkey.sign_schnorr(sighash)

# Add witness to transaction
spending_tx.wit.vtxinwit.append(CTxInWitness([sig]))

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")

#### _Programming Exercise 2.2.13:_ Generate segwit v1 address for a pay-to-contract public key

In [ ]:
# Generate a key pair
privkey, pubkey = generate_bip340_key_pair()

# Generate the pay-to-contract tweak
# Hint: Use tagged_hash("TapTweak", P + bytes)
contract_bytes = "Alice pays 10 BTC to Bob".encode('utf-8')
tweak = int_or_bytes(tagged_hash("TapTweak", pubkey.get_bytes() + contract_bytes))
tweak_private, tweak_point = generate_key_pair(tweak)

# Tweak Alice's key pair with the pay-to-contract tweak
tweaked_pubkey = pubkey + tweak_point
tweaked_privkey = privkey + tweak_private

# Generate the segwit v1 address
tweaked_pubkey_data = tweaked_pubkey.get_bytes()
tweaked_pubkey_program = tweaked_pubkey_data
version = 1
address = program_to_witness(version, tweaked_pubkey_program)
print("Address encoding the segwit v1 output: ", address)